# TensorBoard可视化完全指南

## 核心概念

TensorBoard是TensorFlow的可视化工具包，提供强大的训练过程监控和分析功能：

### 主要功能

1. **Scalars**: 标量数据可视化(损失、准确率等)
2. **Graphs**: 计算图结构可视化
3. **Distributions**: 权重和激活值分布
4. **Histograms**: 张量直方图统计
5. **Images**: 图像数据展示
6. **Embeddings**: 高维数据降维可视化
7. **HParams**: 超参数调优实验对比

## 应用场景

1. **训练监控**: 实时查看训练和验证指标
2. **模型调试**: 检查权重更新和梯度流动
3. **性能分析**: 定位训练瓶颈
4. **超参数优化**: 对比不同配置的效果
5. **结果展示**: 生成专业的训练报告

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import datetime
import shutil

## 1. 准备数据集

使用IMDB电影评论情感分类数据集。

In [ ]:
# 加载IMDB数据集
max_features = 10000  # 词汇表大小
maxlen = 200          # 序列最大长度

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)

# 填充序列到相同长度
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

print(f'训练集形状: {x_train.shape}, 标签形状: {y_train.shape}')
print(f'测试集形状: {x_test.shape}, 标签形状: {y_test.shape}')
print(f'\n训练样本示例:')
print(f'序列: {x_train[0][:20]}...')
print(f'标签: {y_train[0]} (0=负面, 1=正面)')

## 2. 构建模型

构建一个用于文本分类的神经网络。

In [ ]:
def create_model():
    """
    创建文本分类模型
    
    架构:
    - Embedding层: 将词索引转换为稠密向量
    - Conv1D层: 提取局部特征
    - GlobalMaxPooling1D: 池化操作
    - Dense层: 全连接分类
    """
    model = keras.Sequential([
        layers.Embedding(max_features, 128, input_length=maxlen, name='embedding'),
        layers.Conv1D(32, 7, activation='relu', name='conv1d_1'),
        layers.MaxPooling1D(5, name='pool_1'),
        layers.Conv1D(32, 7, activation='relu', name='conv1d_2'),
        layers.GlobalMaxPooling1D(name='global_pool'),
        layers.Dense(64, activation='relu', name='dense_1'),
        layers.Dropout(0.5, name='dropout'),
        layers.Dense(1, activation='sigmoid', name='output')
    ], name='text_classifier')
    
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

model = create_model()
model.summary()

## 3. 配置TensorBoard回调

### TensorBoard关键参数

- `log_dir`: 日志保存目录
- `histogram_freq`: 记录权重直方图的频率(epoch)
- `write_graph`: 是否记录计算图
- `write_images`: 是否记录权重为图像
- `update_freq`: 更新频率('epoch'或batch数)
- `profile_batch`: 性能分析的batch范围
- `embeddings_freq`: 记录嵌入层的频率

In [ ]:
# 创建基础日志目录
base_log_dir = 'tensorboard_logs'
os.makedirs(base_log_dir, exist_ok=True)

# 使用时间戳创建唯一的日志目录
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join(base_log_dir, f'run_{current_time}')

print(f'TensorBoard日志目录: {log_dir}')

# 配置TensorBoard回调
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,         # 每个epoch记录直方图
    write_graph=True,         # 记录计算图
    write_images=False,       # 不记录权重图像(节省空间)
    update_freq='epoch',      # 按epoch更新
    profile_batch='10,20',    # 分析第10-20个batch的性能
    embeddings_freq=1         # 每个epoch记录嵌入层
)

print('TensorBoard回调配置完成')

## 4. 训练模型并记录日志

使用TensorBoard回调训练模型。

In [ ]:
print('\n开始训练模型...')
print(f'日志将保存到: {log_dir}\n')

history = model.fit(
    x_train, y_train,
    epochs=3,  # 测试用，实际训练可设为20
    batch_size=128,
    validation_split=0.2,
    callbacks=[tensorboard_callback],
    verbose=1
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'\n测试结果 - 损失: {test_loss:.4f}, 准确率: {test_acc:.4f}')

## 5. 自定义标量记录

除了自动记录的指标外，还可以记录自定义标量。

In [ ]:
class CustomMetricsCallback(keras.callbacks.Callback):
    """
    自定义指标记录回调
    
    记录额外的训练指标到TensorBoard
    """
    
    def __init__(self, log_dir):
        super().__init__()
        self.log_dir = log_dir
        self.file_writer = tf.summary.create_file_writer(os.path.join(log_dir, 'custom_metrics'))
    
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        
        # 计算训练-验证差距(过拟合指标)
        train_val_gap = logs.get('accuracy', 0) - logs.get('val_accuracy', 0)
        
        # 计算相对改善率
        if epoch > 0 and hasattr(self, 'prev_val_loss'):
            val_loss = logs.get('val_loss', 0)
            improvement = (self.prev_val_loss - val_loss) / self.prev_val_loss * 100
        else:
            improvement = 0
        
        self.prev_val_loss = logs.get('val_loss', 0)
        
        # 记录自定义指标
        with self.file_writer.as_default():
            tf.summary.scalar('train_val_gap', train_val_gap, step=epoch)
            tf.summary.scalar('val_loss_improvement_pct', improvement, step=epoch)
            
            # 记录学习率
            lr = float(keras.backend.get_value(self.model.optimizer.learning_rate))
            tf.summary.scalar('learning_rate', lr, step=epoch)
    
    def on_train_end(self, logs=None):
        self.file_writer.close()

# 使用自定义回调重新训练
print('\n使用自定义指标重新训练...')
custom_log_dir = os.path.join(base_log_dir, f'custom_{current_time}')

model_custom = create_model()
custom_metrics_callback = CustomMetricsCallback(custom_log_dir)

tensorboard_callback_custom = keras.callbacks.TensorBoard(
    log_dir=custom_log_dir,
    histogram_freq=1,
    update_freq='epoch'
)

history_custom = model_custom.fit(
    x_train, y_train,
    epochs=3,  # 测试用
    batch_size=128,
    validation_split=0.2,
    callbacks=[tensorboard_callback_custom, custom_metrics_callback],
    verbose=1
)

print(f'\n自定义指标日志保存到: {custom_log_dir}')

## 6. 对比多个实验

训练多个模型并在TensorBoard中对比。

In [ ]:
def train_with_config(config_name, learning_rate, batch_size, epochs=3):
    """
    使用指定配置训练模型
    """
    print(f'\n训练配置: {config_name}')
    print(f'  学习率: {learning_rate}')
    print(f'  批次大小: {batch_size}\n')
    
    # 创建模型
    model = create_model()
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # 配置日志目录
    exp_log_dir = os.path.join(base_log_dir, 'experiments', config_name)
    
    # 训练模型
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=[
            keras.callbacks.TensorBoard(
                log_dir=exp_log_dir,
                histogram_freq=1
            )
        ],
        verbose=0
    )
    
    # 评估
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f'测试准确率: {test_acc:.4f}')
    
    return history, test_acc

# 进行对比实验
print('========== 对比实验 ==========')

experiments = [
    ('lr_0.001_bs_128', 0.001, 128),
    ('lr_0.0001_bs_128', 0.0001, 128),
    ('lr_0.001_bs_64', 0.001, 64)
]

results = {}
for config_name, lr, bs in experiments:
    history, test_acc = train_with_config(config_name, lr, bs, epochs=3)
    results[config_name] = test_acc

# 显示对比结果
print('\n========== 实验结果对比 ==========')
for config_name, test_acc in results.items():
    print(f'{config_name}: {test_acc:.4f}')

best_config = max(results, key=results.get)
print(f'\n最佳配置: {best_config} (准确率: {results[best_config]:.4f})')

## 7. 记录图像数据

演示如何在TensorBoard中记录图像。

In [ ]:
# 使用MNIST数据集演示图像记录
(mnist_train_images, mnist_train_labels), _ = keras.datasets.mnist.load_data()

# 归一化并添加通道维度
mnist_train_images = mnist_train_images[:100].astype('float32') / 255.0
mnist_train_images = np.expand_dims(mnist_train_images, -1)

# 创建图像日志
image_log_dir = os.path.join(base_log_dir, f'images_{current_time}')
file_writer = tf.summary.create_file_writer(image_log_dir)

# 记录前25张图像
with file_writer.as_default():
    tf.summary.image(
        'MNIST训练样本',
        mnist_train_images[:25],
        max_outputs=25,
        step=0
    )

file_writer.close()
print(f'图像已记录到: {image_log_dir}')

## 8. 超参数调优可视化

使用HParams插件进行超参数实验对比。

In [ ]:
from tensorboard.plugins.hparams import api as hp

# 定义超参数空间
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.0001]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.3, 0.5]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'rmsprop']))

# 定义评估指标
METRIC_ACCURACY = 'accuracy'

hparams_log_dir = os.path.join(base_log_dir, 'hparams_tuning')

with tf.summary.create_file_writer(hparams_log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_LEARNING_RATE, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )

def train_hparams(hparams, run_dir):
    """
    使用指定超参数训练模型
    """
    # 构建模型
    model = keras.Sequential([
        layers.Embedding(max_features, 128, input_length=maxlen),
        layers.Conv1D(32, 7, activation='relu'),
        layers.GlobalMaxPooling1D(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(hparams[HP_DROPOUT]),
        layers.Dense(1, activation='sigmoid')
    ])
    
    # 编译模型
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # 训练
    model.fit(
        x_train, y_train,
        epochs=2,  # 快速测试
        batch_size=128,
        validation_split=0.2,
        verbose=0
    )
    
    # 评估
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    
    return accuracy

# 运行超参数搜索
print('\n========== 超参数调优 ==========')
session_num = 0

for learning_rate in HP_LEARNING_RATE.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_LEARNING_RATE: learning_rate,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer
            }
            
            run_name = f'run-{session_num}'
            print(f'{run_name}: {hparams}')
            
            run_dir = os.path.join(hparams_log_dir, run_name)
            
            # 训练并记录结果
            with tf.summary.create_file_writer(run_dir).as_default():
                hp.hparams(hparams)  # 记录超参数
                accuracy = train_hparams(hparams, run_dir)
                tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
                print(f'  准确率: {accuracy:.4f}\n')
            
            session_num += 1

print(f'超参数调优日志保存到: {hparams_log_dir}')

## 9. 启动TensorBoard

生成启动TensorBoard的命令。

In [ ]:
print('\n========== 启动TensorBoard ==========')
print(f'\n在终端中运行以下命令启动TensorBoard:\n')
print(f'tensorboard --logdir={base_log_dir}\n')
print(f'然后在浏览器中访问: http://localhost:6006/\n')
print('TensorBoard功能说明:')
print('  - Scalars: 查看损失和准确率曲线')
print('  - Graphs: 查看模型计算图')
print('  - Distributions: 查看权重分布')
print('  - Histograms: 查看张量直方图')
print('  - HParams: 对比超参数实验')
print('  - Images: 查看记录的图像')
print('  - Time Series: 查看时间序列数据')

# 也可以在Jupyter中内嵌TensorBoard
print('\n在Jupyter Notebook中，可以使用以下命令内嵌TensorBoard:')
print(f'%load_ext tensorboard')
print(f'%tensorboard --logdir {base_log_dir}')

## 10. 日志目录结构说明

In [ ]:
def print_directory_tree(path, prefix='', max_depth=3, current_depth=0):
    """
    打印目录树结构
    """
    if current_depth >= max_depth:
        return
    
    try:
        entries = sorted(os.listdir(path))
        for i, entry in enumerate(entries[:10]):  # 限制显示数量
            entry_path = os.path.join(path, entry)
            is_last = (i == len(entries) - 1) or (i == 9)
            
            connector = '└── ' if is_last else '├── '
            print(f'{prefix}{connector}{entry}')
            
            if os.path.isdir(entry_path):
                extension = '    ' if is_last else '│   '
                print_directory_tree(entry_path, prefix + extension, max_depth, current_depth + 1)
        
        if len(entries) > 10:
            print(f'{prefix}    ... 还有{len(entries) - 10}个文件/目录')
    except PermissionError:
        pass

print('\n========== TensorBoard日志目录结构 ==========')
print(f'{base_log_dir}/')
print_directory_tree(base_log_dir, max_depth=2)

## 11. 清理日志文件

In [ ]:
# 清理测试产生的日志
if os.path.exists(base_log_dir):
    shutil.rmtree(base_log_dir)
    print(f'已删除日志目录: {base_log_dir}')

print('清理完成')

## 总结

### TensorBoard最佳实践

1. **日志管理**:
   - 使用时间戳或实验名称区分不同运行
   - 定期清理旧日志节省空间
   - 使用子目录组织多个实验

2. **性能优化**:
   - `histogram_freq`不要设置太小(建议>=1)
   - 大规模训练时考虑降低记录频率
   - 使用`profile_batch`限制性能分析范围

3. **可视化技巧**:
   - 使用平滑功能查看长期趋势
   - 对比模式下可同时查看多个实验
   - 使用正则表达式过滤显示的图表

4. **自定义记录**:
   ```python
   with file_writer.as_default():
       tf.summary.scalar('custom_metric', value, step=epoch)
       tf.summary.histogram('layer_weights', weights, step=epoch)
       tf.summary.image('predictions', images, step=epoch)
   ```

### 常见问题

1. **TensorBoard不更新**:
   - 检查日志目录是否正确
   - 刷新浏览器或重启TensorBoard
   - 确认回调函数已添加到训练中

2. **日志文件过大**:
   - 降低`histogram_freq`
   - 禁用`write_images`
   - 减少`embeddings_freq`

3. **性能分析失败**:
   - 检查`profile_batch`设置
   - 确保GPU驱动支持性能分析
   - 尝试减小分析的batch范围

### 高级功能

1. **Profiler性能分析**:
   - 识别训练瓶颈
   - 优化数据管道
   - GPU利用率分析

2. **What-If Tool**:
   - 可视化模型预测
   - 分析模型公平性
   - 交互式探索特征影响

3. **Embedding Projector**:
   - 高维向量可视化
   - 支持PCA、t-SNE、UMAP降维
   - 交互式搜索相似向量

### 命令行技巧

```bash
# 基础启动
tensorboard --logdir=logs

# 指定端口
tensorboard --logdir=logs --port=6007

# 绑定所有网络接口(允许远程访问)
tensorboard --logdir=logs --host=0.0.0.0

# 启用调试模式
tensorboard --logdir=logs --debugger_port=6064

# 限制缓存大小
tensorboard --logdir=logs --max_reload_threads=1
```

### 集成其他工具

- **Weights & Biases**: 云端训练监控
- **MLflow**: 实验跟踪和模型管理
- **Neptune.ai**: 协作实验管理
- **Comet.ml**: 自动实验跟踪

### 参考资源

- TensorBoard官方文档: https://www.tensorflow.org/tensorboard
- GitHub仓库: https://github.com/tensorflow/tensorboard
- 教程和示例: https://www.tensorflow.org/tensorboard/get_started